In [3]:
cd cities

/cfs/klemming/nobackup/l/lxin/support/jupyter-lab/cities


In [16]:
# prepare ipyparallel

import ipyparallel as ipp
client = ipp.Client()
dview = client[:]

In [17]:
# prepare data as a list of tuples(latitude_1, latitude_2, longitude_1, longitude_2)

import dist_cities as dc
cities = dc.read_cities()
coord_pairs = dc.create_coord_pairs(cities)

In [21]:
import math

def calc_dist(coord_pair):

    """Calculate the distance from a coordinate pair (latitude_1, latitude_2,
    longitude_1, longitude_2)"""

    p1 = coord_pair[0] / 180.0 * math.pi
    p2 = coord_pair[1] / 180.0 * math.pi

    cp1 = math.cos(p1)
    cp2 = math.cos(p2)

    dp = p2 - p1

    dL = (coord_pair[2] - coord_pair[3]) / 180.0 * math.pi

    a = math.sin(dp/2) **2 + cp1 * cp2 * math.sin(dL/2) **2
    c = 2.0 * math.atan2(math.sqrt(a), math.sqrt(1.0 - a))

    R = 6371 # radius of Earth in km

    return R*c

In [22]:
dview.execute('import math')

<AsyncResult: execute>

In [23]:
%%timeit
output = dview.map_sync(calc_dist, coord_pairs)
print(max(output))

19983.405422311647
19983.405422311647
19983.405422311647
19983.405422311647
19983.405422311647
19983.405422311647
19983.405422311647
19983.405422311647
657 ms ± 23.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [24]:
%%timeit
output = [calc_dist(p) for p in coord_pairs]
print(max(output))

19983.405422311647
19983.405422311647
19983.405422311647
19983.405422311647
19983.405422311647
19983.405422311647
19983.405422311647
19983.405422311647
804 ms ± 2.16 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
